## Sentinel 3 Data Streaming

This service offers compressed Sentinel 3 NDVI data and makes it available via a high quality and memory efficient streaming solution.


## Prerequisites
### DestinE Platform Credentials

You need to have an account on the [Destination Earth Platform](https://auth.destine.eu/realms/desp/account).

#### ⚠️ Warning: Authorized Access Only
The usage of this notebook and data access is reserved only to authorized user groups.

## Access the Data
With a DESP account you can access the stream data proposed in this tutorial.

In [ ]:
%%capture cap
%run ./auth.py

In [ ]:
output = cap.stdout.split('\n')
refresh_token = output[1]
token = output[2]
print(token)

# Imports and general definitions
We start by importing necessary packages and definitions regarding the resolution and the endpoint to the streaming api.

Note: The API token must be set here including the user group. This happens in **Authentification**.

In [ ]:
from datetime import datetime
import xarray as xa
import rioxarray  # noqa
from rasterio.transform import from_bounds
from dtelib_s3 import DTEStreamer_S3
import gc

FORMAT = '%Y-%m-%dT%H:%M'

# Parameters for stream access

Here the parameters are set to access the data from the service.

*variable*: an abbreviated variable for the data</br>
*program_subset*: a name for the program subest for the data</br>
*start_date*: the time and date to start the stream</br>
*end_date*: the time and date to end the stream</br>
</br>

To select a stream, chose parameter values from the table above, or if you have a *code snippet*, use it to replace the code in the cell below:

In [ ]:
continent = "Europe"

start_date = datetime.strptime("2021-01-01T00:00", FORMAT)
end_date = datetime.strptime("2022-03-01T00:00", FORMAT)

save_path = "./geotiffs"

if not os.path.exists(save_path):
    os.makedirs(save_path, exist_ok=True)

# Loading the stream

With the DTEStreamer class we can easily access the data stream through the api and access individual data frames. 
At first, we create a DTEStreamer object with the parameters defined in the step above. The object initializes by calling the api to get meta information about the stream and the location of the stream. (You can take a look at the api yourself in the swagger [here](https://streamer.destine.eu/api/v1/)). Also, ffmpeg is used to seek to the first image of the stream, according to *start_date*.

The load_next_image() method then loads the next image into a numpy array, which is stored in the list *time_series* for further use.

Note that in this example, the data and time stamps are loaded into a list. A print statements keep us on track with the progress.

Note: This example should be modified to your purpose especially if you plan to do a long time series analysis, as it will load all the data of the loop into memory.

In [ ]:
streamer = DTEStreamer_S3(continent=continent, start_date=start_date, end_date=end_date, token=token)

for image, image_info in streamer.load_next_image():

    da = xa.DataArray(image,
                      dims=['latitude', 'longitude'],
                      name="NDVI"
                     )

    transform = from_bounds(*streamer.bbox(), streamer.nx(), streamer.ny())

    da = da.rio.write_crs("EPSG:4326")
    da = da.rio.write_transform(transform)
    
    ft_stamp = image_info["img_date"].strftime("%Y%m%d")
    satellite = image_info["satellite"]
    
    tif_name = f'{save_path}/NDVI_{continent}_{satellite}_{ft_stamp}.tif'
    da.rio.to_raster(tif_name, num_threads='all_cpus')
    print(tif_name)
    
    del da
    gc.collect()

    
print("done")